# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner and combine it with our other data skills. In this lab you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer follow-up questions. 


### Outline:

1. Determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create Python functions to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

*Something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data*

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- What are the 5 businesses with the highest average ratings?
- What are the 5 businesses with the lowest average ratings?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- How many businesses have a rating greater than or  4.5?
- How many businesses have a rating less than 3?
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [1]:
## Connect to DB server on AWS
import mysql.connector
from mysql.connector import errorcode

In [13]:
## Create new DB 
db_name = 'Yelp_Reviews'

In [97]:
import config

In [98]:
config.host

'kyledb.cyfsqcs7xvaz.us-east-2.rds.amazonaws.com'

In [15]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd
)

print(cnx)

In [16]:
cur = cnx.cursor()

In [7]:
def create_database(cur, database):
    try:
        cur.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cur.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cur, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

Database Yelp_Reviews does not exists.
Database Yelp_Reviews created successfully.


In [8]:
cur.close()
cnx.close()

In [9]:
# Create a table for the Businesses
db_name = 'Yelp_Reviews'

In [10]:
tables = {}
tables['businesses'] =(
     """CREATE TABLE businesses (
        id char(100) NOT NULL,
        name char(50) NOT NULL,
        rating float(10),
        price char(10),
        reviews_count int(100),
        zip int(10),
        city char(50),
        phone char(50),
        PRIMARY KEY (id))"""
)

In [11]:
tables['businesses']

'CREATE TABLE businesses (\n        id char(100) NOT NULL,\n        name char(50) NOT NULL,\n        rating float(10),\n        price char(10),\n        reviews_count int(100),\n        zip int(10),\n        city char(50),\n        phone char(50),\n        PRIMARY KEY (id))'

In [12]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cur = cnx.cursor()

In [13]:
table_description = tables['businesses']
cur.execute(table_description)

In [14]:
# Create a table for the reviews
cnx.commit()


In [15]:
cur.close()
cnx.close()

## Part 2: Create ETL pipeline for the business data from the API

In [19]:
# write a function to make a call to the DB
import json
import requests

In [20]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)
cur = cnx.cursor()

In [5]:
client_id = "9J_ck2JO3fi6RH5JLu8A-A"
api_key = "viVSrhO9Q0gaHiDJCaGZMUlypHjZRjgi5D6DIcQbcjxpkBbHncF1QEI48CNtGPiKpCoCO06tp6SnnzzWUMXwh6Sz-mFRT1Z0a5iD3L0n-D57m2hwBXxaNJOwZHh6XXYx"


In [19]:
term = 'Pizza'
location = 'New York City'
SEARCH_LIMIT = 50

url = 'https://api.yelp.com/v3/businesses/search' #endpoint 

headers = {
        'Authorization': 'Bearer {}'.format(api_key), # need to pass along the API key, this is how it asks for authorization
    }
#when you make the request, it will look into the header for this information 

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                
            }

response = requests.get(url, headers=headers, params=url_params)


print(response)
print(type(response.text))
print(response.text[:500])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "ysqgdbSrezXgVwER2kQWKA", "alias": "julianas-pizza-brooklyn-5", "name": "Juliana's Pizza", "image_url": "https://s3-media1.fl.yelpcdn.com/bphoto/7JtwTxhWHf3YS70Ss_CfxA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/julianas-pizza-brooklyn-5?adjust_creative=9J_ck2JO3fi6RH5JLu8A-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9J_ck2JO3fi6RH5JLu8A-A", "review_count": 2071, "categories": [{"alias": "pizza", "title": "Pizza"}], "rating": 4.


In [68]:
def get_restaurants(offset):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
    url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset
            }
    response = requests.get(url, headers=headers, params=url_params)
    return response

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [69]:
response.json()['total']

10900

In [70]:
get_restaurants(0)

<Response [200]>

In [71]:
def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        cur += 50
    return results

In [72]:
business_data = all_results(url_params, api_key)

10900 total matches found.


In [73]:
len(business_data)

20

In [74]:
business_data = [val for sublist in business_data for val in sublist]

In [75]:
business_data[0]

{'id': 'WIhm0W9197f_rRtDziq5qQ',
 'alias': 'lombardis-pizza-new-york-4',
 'name': "Lombardi's Pizza",
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_hobp1KpYDPOHNoCKw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/lombardis-pizza-new-york-4?adjust_creative=9J_ck2JO3fi6RH5JLu8A-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=9J_ck2JO3fi6RH5JLu8A-A',
 'review_count': 5960,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.7215934960083, 'longitude': -73.9955956044561},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '32 Spring St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10012',
  'country': 'US',
  'state': 'NY',
  'display_address': ['32 Spring St', 'New York, NY 10012']},
 'phone': '+12129417994',
 'display_phone': '(212) 941-7994',
 'distance': 1798.9959778083735}

In [76]:
list_of_restaurants = []
for business in business_data:
    restaurant_info= {}
    for key in business: 
        restaurant_info['id']= business.get('id')
        restaurant_info['name']= business.get('name')
        restaurant_info['rating']= business.get('rating')
        restaurant_info['price']= business.get('price')
        restaurant_info['reviews_count']= business.get('review_count')
        for i in business['location']:
            restaurant_info['zip']= business['location']['zip_code']
            restaurant_info['city']= business['location']['city']
        restaurant_info['phone']= business.get('phone')
    list_of_restaurants.append(restaurant_info)
       
            
list_of_restaurants[0:100]

[{'id': 'WIhm0W9197f_rRtDziq5qQ',
  'name': "Lombardi's Pizza",
  'rating': 4.0,
  'price': '$$',
  'reviews_count': 5960,
  'zip': '10012',
  'city': 'New York',
  'phone': '+12129417994'},
 {'id': 'zj8Lq1T8KIC5zwFief15jg',
  'name': 'Prince Street Pizza',
  'rating': 4.5,
  'price': '$',
  'reviews_count': 3084,
  'zip': '10012',
  'city': 'New York',
  'phone': '+12129664100'},
 {'id': 'ysqgdbSrezXgVwER2kQWKA',
  'name': "Juliana's Pizza",
  'rating': 4.5,
  'price': '$$',
  'reviews_count': 2071,
  'zip': '11201',
  'city': 'Brooklyn',
  'phone': '+17185966700'},
 {'id': 'v1DHGRNCH9247WLYoaoA9A',
  'name': "L'industrie Pizzeria",
  'rating': 5.0,
  'price': '$',
  'reviews_count': 471,
  'zip': '11211',
  'city': 'Brooklyn',
  'phone': '+17185990002'},
 {'id': 'nUpA3sY5b6dynC7aj0eV6A',
  'name': "Rizzo's Fine Pizza",
  'rating': 4.5,
  'price': '$$',
  'reviews_count': 708,
  'zip': '10002',
  'city': 'New York',
  'phone': '+16464541262'},
 {'id': 'Zjj8JkJm0yRyrUFffr7Eww',
  'name

In [77]:
add_businesses = ("INSERT INTO businesses "
               "(id, name, rating, price, reviews_count, zip, city, phone) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")

In [78]:
business_list = []
for bus in list_of_restaurants:
    business_tuple= (bus['id'], bus['name'], bus['rating'], bus['price'], bus['reviews_count'], bus['zip'], 
                     bus['city'], bus['phone'])
    business_list.append(business_tuple)

In [3]:
import config

In [4]:
import mysql.connector
from mysql.connector import errorcode

In [81]:
print(config.host)

kyledb.cyfsqcs7xvaz.us-east-2.rds.amazonaws.com


In [82]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)

cur = cnx.cursor()

In [83]:
cur.close()
cnx.close()

In [84]:
add_businesses

'INSERT INTO businesses (id, name, rating, price, reviews_count, zip, city, phone) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'

In [85]:
business_list[0]

('WIhm0W9197f_rRtDziq5qQ',
 "Lombardi's Pizza",
 4.0,
 '$$',
 5960,
 '10012',
 'New York',
 '+12129417994')

In [ ]:
for i in range(len(list_of_restaurants)):
    if list_of_restaurants[i]['id'] == 'IZBcd_82bcXEB25ZEyxg7w':
        del list_of_restaurants[i]
        break

In [ ]:
for i in list_of_restaurants:
    if i['id'] == '_WttOSi2_ggdvUI6GvaqTg':
        print(i)

In [ ]:
for i in range(len(list_of_restaurants)):
    if list_of_restaurants[i]['id'] == 'wvn-FO-tY9_iWmA_OgMvEA':
        del list_of_restaurants[i]
        break

In [ ]:
for i in list_of_restaurants:
    if i['id'] == 'wvn-FO-tY9_iWmA_OgMvEA':
        print(i)

In [86]:
clean_list = [] 
for i in range(len(list_of_restaurants)): 
    if list_of_restaurants[i] not in list_of_restaurants[i + 1:]: 
        clean_list.append(list_of_restaurants[i]) 

In [87]:
clean_list[0]

{'id': 'WIhm0W9197f_rRtDziq5qQ',
 'name': "Lombardi's Pizza",
 'rating': 4.0,
 'price': '$$',
 'reviews_count': 5960,
 'zip': '10012',
 'city': 'New York',
 'phone': '+12129417994'}

In [ ]:
for i in clean_list:
    if i['id'] == 'wvn-FO-tY9_iWmA_OgMvEA':
        print(i)

In [88]:
business_list = []
for bus in clean_list:
    business_tuple= (bus['id'], bus['name'], bus['rating'], bus['price'], bus['reviews_count'], bus['zip'], 
                     bus['city'], bus['phone'])
    business_list.append(business_tuple)

In [89]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)

cur = cnx.cursor()

In [90]:
for i in range(0, len(business_list), 50):
    print(i)     
    cur.executemany(add_businesses, business_list[i:i+50])
    cnx.commit()

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [91]:
cur.close()
cnx.close()

In [ ]:
for i in list_of_restaurants:
    if i['id'] == 'IZBcd_82bcXEB25ZEyxg7w':
        print(i)

In [ ]:
for i in range(len(list_of_restaurants)):
    if list_of_restaurants[i]['id'] == 'IZBcd_82bcXEB25ZEyxg7w':
        del list_of_restaurants[i]
        break

In [ ]:
for i in list_of_restaurants:
    if i['id'] == 'IZBcd_82bcXEB25ZEyxg7w':
        print(i)

In [ ]:
business_list[0]

In [ ]:
for i in list_of_restaurants:
    if i['id'] == '_WttOSi2_ggdvUI6GvaqTg':
        print(i)
        


In [ ]:
for i in list_of_restaurants:
    if list_of_restaurants[i]['id'] == 'IZBcd_82bcXEB25ZEyxg7w':
        del list_of_restaurants[i]
        break

In [ ]:
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        results += 50
    return df


url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

In [ ]:
new_data = all_results(url_params, api_key)

In [ ]:
print(response.status_code)

In [ ]:
response.json().keys()

In [ ]:
for key in response.json().keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n\n')

In [ ]:
yelp_pizza_data =response.json()

In [ ]:
yelp_pizza_data['total']

In [ ]:
list_of_restaurants = []
for business in yelp_pizza_data['businesses']:
    restaurant_info= {}
    for key in business: 
        restaurant_info['id']= business.get('id')
        restaurant_info['name']= business.get('name')
        restaurant_info['rating']= business.get('rating')
        restaurant_info['price']= business.get('price')
        restaurant_info['reviews_count']= business.get('review_count')
        for i in business['location']:
            restaurant_info['zip']= business['location']['zip_code']
            restaurant_info['city']= business['location']['city']
        restaurant_info['phone']= business.get('phone')
    list_of_restaurants.append(restaurant_info)
       
            
list_of_restaurants

In [ ]:
import mysql.connector
from mysql.connector import errorcode

In [ ]:
add_businesses = ("INSERT INTO businesses "
               "(id, name, city, zip, reviews_count, favoritefood) "
               "VALUES (%s, %s, %s, %s, %s, %s)")
    

In [ ]:
cursor.executemany(add_students, list_of_restaurants)
cnx.commit()
cursor.close()
cnx.close()

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB



In [ ]:
# Write a function to take your parsed data and insert it into the DB

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [ ]:
# write a query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews
# then parse out the relevant information

In [ ]:
# write a function to insert the parsed data into the reviews table

## Part 4: Write SQL queries that will answer the questions posed. 

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter.***

In [20]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.append(yelp_call(url_params, api_key))
        time.sleep(1) #Wait a second
        results += 50
    return df

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

10900 total matches found.


TypeError: 'int' object is not iterable

In [28]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.passwd,
    database = db_name
)

cur = cnx.cursor()


In [29]:
cur.execute("""SELECT id FROM businesses;""")
business_id = cur.fetchall()

In [153]:
type(business_id[0])

tuple

In [32]:
clean_id = []
for i in business_id: 
    clean = str(i)
    clean_id.append(clean)

super_clean_ids = []
for n in clean_id:
    n = n.strip('"(')
    n = n.strip(')"')
    n = n.strip(',')
    n = n.strip("'")
    super_clean_ids.append(n)
    

In [ ]:
super_clean_ids[5]

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

In [8]:
import requests 

In [21]:

 #endpoint 

headers = {
        'Authorization': 'Bearer {}'.format(api_key), # need to pass along the API key, this is how it asks for authorization
    }
#when you make the request, it will look into the header for this information 
response = requests.get(url, headers=headers)



In [34]:
# def yelp_call(id_list, api_key):
#     url = 'https://api.yelp.com/v3/businesses/{id}/reviews'
#     headers = {'Authorization': 'Bearer {}'.format(api_key)}
#     response = requests.get(url, headers=headers)
    
#     data = response.json()['reviews']
#     return data

location = 'Astoria NY'
def get_reviews(id_list, api_key):
    for i in id_list:
        url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(i)
        headers = {
        'Authorization': 'Bearer {}'.format(api_key),
            }
        
        url_params = {'location': location.replace(' ', '+'),
             }
        response = requests.get(url, headers=headers, params)
        data = response.json()
        return data
        




SyntaxError: positional argument follows keyword argument (<ipython-input-34-9717e404bf07>, line 19)

In [23]:
def all_results(id_list, api_key):
    num = response.json()
    print('{} total matches found.'.format(num))
    results = []
    for i in range(0,len(id_list)-1):
        results.append(get_reviews(id_list[i], api_key))
        time.sleep(1) #Wait a second
        results[i] = id_list[i]
    return results

In [30]:
# def get_reviews(id, offset):
#     url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(id)
#     headers = {
#         'Authorization': 'Bearer {}'.format(api_key),
#             }
#     url_params = {
#                 'locale': locale.replace(' ', '+'),
#                 'offset': offset
#             }
#     response = requests.get(url, headers=headers, params=url_params)
#     return response

In [33]:
all_results(super_clean_ids,api_key)

{'error': {'code': 'VALIDATION_ERROR', 'description': 'Please specify a location or a latitude and longitude'}} total matches found.


['-5xLqs3iGFbBkJG3QNh6Bw',
 '-9NO2IDCJMyNGZMAQ1-VQw',
 '-9TisrxCZT_Alm2UV9WVXA',
 '-CCfu19Uuy_kD2t4ZwsQfA',
 '-F0fEtdhbQ4DczCqO-tebQ',
 '-Gc5sTFxoXM3Tiv_dkSJZg',
 '-HQWFXtDoRwu6D3rvH8nTQ',
 '-jdadX3fGXA2TDapnufYFA',
 '-MAZ8nAtKIIIXQ0_SqnXMw',
 '-Td-YEPTOUq2H3LRWgRxUw',
 '-WdZvgWBMYg5fSuav78ZRA',
 '-wG31d9zcEsW_aE14tX4EA',
 '-Zebvg7OAy1YgWRwIw7Ogg',
 '0-hM-ARw7sLpkyyow_qf6w',
 '02sAHm3_CsRKE2IeT5iaPA',
 '02sUerL13Wt4Ii2-kDzlow',
 '03dlQRQNhOKZSD_ZGGUPVQ',
 '04xHcYsqA3jCzpsX1Vs8ZQ',
 '06tnwnEQM2j9A6dC3HC_TQ',
 '096LKIm7omuiwKY3rcUQqA',
 '0B3WbaM1hnn3Xut1bitSyw',
 '0E089bWgJvUl396T693rYg',
 '0k6OrRfbWt5oVPI4JCztlw',
 '0laCKZiHrdM_ZKdhTFbGgg',
 '0ROpM3UcZhprN6dhwK2Hsw',
 '0Tvwk0hvgaY0QVIsZGTZKQ',
 '0UdC0VYX6aAvLiOcpR4MVQ',
 '0UIUQWmwf_yxyygLXPQ0Hw',
 '0xS2Jj-RZuFQKgxEWuVxzA',
 '11NljAW6o286ER8h5oht7w',
 '15d_Y6TVOQFD2KRVkxuAjw',
 '15k7iqFbhf4h8L01yggLqg',
 '16ZnHpuaaBt92XWeJHCC5A',
 '19zArMBG2qW2k16Y0i2aBA',
 '1A2-oW9IRHSzVzo-iouI3g',
 '1d03Uru0tjjSLl1Afe0lkg',
 '1DT6XYMzcekpFQ41Hj44vA',
 